In [60]:
import os
os.chdir("C:\\Users\\vansh\\AI and ML reading material\\MLOps\\New folder\\Project 1-Chest Cancer prediction model")

In [61]:
pwd

'C:\\Users\\vansh\\AI and ML reading material\\MLOps\\New folder\\Project 1-Chest Cancer prediction model'

In [62]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/psycoplankton/Chest-Cancer-Prediction-Model.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "psycoplankton"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "fb3226e715cbe3f9e6ed8fa34a3b954484e5691d"

In the command prompt, write export for linux. Replace export with set for windows command prompt.

export/set MLFLOW_TRACKING_URI=https://dagshub.com/psycoplankton/Chest-Cancer-Prediction-Model.mlflow

export/set MLFLOW_TRACKING_USERNAME=psycoplankton

export/set MLFLOW_TRACKING_PASSWORD=fb3226e715cbe3f9e6ed8fa34a3b954484e5691d

In [63]:
import tensorflow as tf


In [64]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int
    

In [66]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [67]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data/train",
            mlflow_uri="https://dagshub.com/psycoplankton/Chest-Cancer-Prediction-Model.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [68]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [69]:
class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20,
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores ={"loss":self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_score = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            #Model registry does not work with file store
            if tracking_url_type_score != "file":

                #register the model
                #there are other ways to use the Model Registry, which depends on the use case
                #please refer to the doc for more information
                # https://mlflow.org/docs/latest/model-registry.html
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16")

            else:
                mlflow.keras.log_model(self.model, "model")


In [70]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-08-15 14:36:37,792: INFO: common: yaml file: C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\config\config.yaml load successfully]
[2024-08-15 14:36:37,792: INFO: common: yaml file: C:\Users\vansh\AI and ML reading material\MLOps\New folder\Project 1-Chest Cancer prediction model\params.yaml load successfully]
[2024-08-15 14:36:37,803: INFO: common: created directory at :artifacts]
Found 122 images belonging to 4 classes.
8/8 [==============================] - 12s 1s/step - loss: 31.2910 - accuracy: 0.3607
[2024-08-15 14:36:50,824: INFO: common: json file saved at: scores.json]


2024/08/15 14:36:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-15 14:36:53,712: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\vansh\AppData\Local\Temp\tmpxhsnm_cv\model\data\model\assets
[2024-08-15 14:36:54,453: INFO: builder_impl: Assets written to: C:\Users\vansh\AppData\Local\Temp\tmpxhsnm_cv\model\data\model\assets]


c:\Users\vansh\anaconda3\envs\mlops\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16'.
2024/08/15 14:37:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16, version 1
Created version '1' of model 'VGG16'.
